In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random, sys

text = """
'신의 퀴즈' 류덕환이 서유정을 의심했다.
29일 방송된 케이블채널 OCN 수목드라마 '신의 퀴즈:리부트' 6회에서는 요양병원을 찾아간 한진우(류덕환 분), 강경희(윤주희)의 모습이 그려졌다.
이날 사건의 모든 방향이 원장 신연화(서유정)가 있는 요양병원을 가리키고 있단 게 밝혀졌다. 강경희는 윤현종 누나에게 "현종 씨랑 지금 보신 분들 공통점 아냐. 같은 자살 사이트 회원이었고, 이 요양병원을 거쳐 간 후 실종됐다. 이 병원의 누군가와 관련되어 있을 확률이 높다"라고 밝혔다.
한진우는 신연화를 찾아가 "윤현종의 MAO-A에 대한 또 다른 견해가 있었냐. 특이점이나 합병증 여부"라며 "윤현종 씨에게 수술 자국이 있더라"라고 물었다. 이에 신연화는 "저희 병원엔 수술방이 없다. 당연히 뇌 수술도 하지 못한다"라고 말했다.
그러자 한진우는 "어떻게 알았냐. 뇌쪽 수술이란 거. 전 말씀 드린 적 없는데. 게다가 MAO-A는 뇌 수술과 직접적인 연관도 없고"라고 의심했다.
"""

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
print('사용되고 있는 문자의 수:',  chars )

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# 모델 구축하기(LSTM)
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)

        #텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Using TensorFlow backend.


사용되고 있는 문자의 수: 163
사용되고 있는 문자의 수: ['\n', ' ', '"', "'", '(', ')', ',', '-', '.', '2', '6', '9', ':', 'A', 'C', 'M', 'N', 'O', '가', '간', '강', '같', '거', '건', '게', '견', '경', '고', '공', '과', '관', '국', '군', '그', '금', '나', '날', '냐', '널', '높', '뇌', '누', '는', '다', '단', '당', '대', '더', '덕', '데', '도', '됐', '되', '된', '드', '든', '들', '떻', '또', '라', '란', '랑', '러', '려', '련', '류', '률', '른', '를', '리', '린', '마', '말', '며', '모', '목', '못', '물', '밝', '방', '병', '보', '부', '분', '블', '사', '살', '서', '송', '수', '술', '습', '신', '실', '심', '씀', '씨', '아', '알', '았', '양', '어', '없', '었', '에', '엔', '여', '연', '와', '요', '우', '원', '유', '윤', '은', '을', '의', '이', '인', '일', '있', '자', '장', '저', '적', '전', '점', '접', '정', '졌', '종', '주', '즈', '증', '지', '직', '진', '쪽', '찾', '채', '쳐', '케', '퀴', '키', '통', '트', '특', '하', '한', '합', '해', '했', '향', '혀', '현', '혔', '화', '확', '환', '회', '후', '희', '히']
학습할 구문의 수: 158
텍스트를 ID 벡터로 변환합니다...

--------------------------------------------------
반복 = 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Epoch 1/1
158/158 [==============================] - 0s 2ms/step - loss: 5.0523

--- 다양성 =  0.2
--- 시드 = "는 뇌 수술과 직접적인 연관도 없고""
는 뇌 수술과 직접적인 연관도 없고"                                                                                                                                                                                                                                                                                                                                                                                                                

--- 다양성 =  0.5
--- 시드 = "는 뇌 수술과 직접적인 연관도 없고""
는 뇌 수술과 직접적인 연관도 없고"      '                                                .                .                                 밝                                                            밝                          .                     .             .                     .         .                    밝                                                                                        .        다

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


...

이날날는건는는는는는는는 거현 누 병는 퀴리"현  윤현  씨는 밝현현 누 병는 모현는 씨저병병는 모현는 모어는 밝윤현  병병는 거저현병 병는 수는는 모저병 병회는는는는양는는 거현 누병는 모어는 밝윤현  누가 는 퀴현는 모저현  한신는 퀴현는 모양됐는 강윤현는 모신화는 뇌현 수술도 모신적는 M신는는는는는는 모저병방 회는는는는양원을 거현 누병는 모어는 밝윤현  병 는 거현는 누 병는는는양는양는는 거현 씨  병원을 거리 간  분는 공는 밝어며 강윤현는 M신화는 퀴현는 모신는 수목도 뇌병수는 모어는  한는 는 수목방거부 간'는 퀴현는 모어"며 강윤현는 씨저병O-에에는는는는는는는양는 거현 병 병 원을 거었다. 강윤는 는는는는 모저방병 병한진는는는양어을 거현 병병는 모어는 밝윤현  누는 는현는 밝어화는 모현는 씨저병병한는

--- 다양성 =  0.5
--- 시드 = "경희(윤주희)의 모습이 그려졌다.
이"
경희(윤주희)의 모습이 그려졌다.
이날병는는는는는는는는 모양방을 밝는 밝어방는 말라며 뇌현 강는 는현는 모어수) 씨다. 강신는는는는는는는 모양방원을 거현 )"현  윤현며 누저게 M 병는는 는는는는양가리는는 모수는 모환)O 강는 는는는는 모현방 누 는 수는는 모)가 는 한는는는는양수목 거리강진는는는양어"현됐 뇌 수술도 모이블는 강는는 모어방는 강O며 수신"는 거현 누 강 는 는현는 모저O 분 분는 모는는 거. 뇌 병한는는는는는 거현됐며 강신는 는현는 밝저O 병 는 퀴심수는 모현방 누 는 는는는는 수술방방국없병는 모어는 밝어증 M윤현병 는는 수술도 없지아 수는는 모)가 수"는 모어블방 강 는 는는는는 모히O-  있는 는 수"공공진히  는 수술도 없있는 는현는 는양밝원을 거심 는 수는 모저방있는 한현는 모"현며 거지 보한신는 는는는는 됐저병병 병는

--- 다양성 =  1.0
--- 시드 = "경희(윤주희)의 모습이 그려졌다.
이"
경희(윤주희)의 모습이 그려졌다.
이날는 신는는 퀴양도는 찾현심 강신었는는찾저현병"한히며윤 병신는란.연화현 M수는 모모는 국는현 도높윤현"됐현  

날99방방방 없이"라며 뇌윤현수  회회회는는 모양가리M  회리수었  없다. 강날9요"어을 거리없간' 회한진는 모양이점방  없한. 라다 말이블이었 한저는 가리가리--에게 한현  씨게 지술방한진 한( 말환 수목방란. M리한-  찾어수 밝란. M은 M M목O--에 한한진  양양원을을거리리간간 한라라 말말술방한진는는 모어수방방 없없한. 강다다. 이날요신방원을 거저 간병  회이었었다. 이날요신원원을 거리가리간   이었도  없지. 강은는 수목방란마 없한는 M어다O M이점 회회퀴었다. 뇌간요수목란란...
날날M건  모심방방방없없"라며 

--- 다양성 =  0.5
--- 시드 = "이날 사건의 모든 방향이 원장 신연화"
이날 사건의 모든 방향이 원장 신연화는 찾저가 병는 원을 거리.간 한 M 씨됐란가리한는 수양말란을리퀴리한는 라""라 말이 수술방란회회.한진었 말"어목을 거리방한  한저말 이점신방 퀴한라며 말신 수목목란마리한는었는양유가심M심했.
뇌9수방방방없없"어며 강윤현  이이점신회회퀴었다. 뇌M수수목란란도..
날날날건건 모환방방방 강없("라환 찾다. 이었신신 .
러 강모현방 누 이저했신.
한진(는 말어 밝을 거리한진 는 밝목 도 없한는 M저가 M 수목란란-.
한진진는 말어원을을거률거한날 말신 화목마마란신화는 퀴저가리는 M심했란.

99있방 누 가리M현  한회화는 한저가 병증 원을 거리연히히   이점도 회지한.한진 말요양원을 거리한간연 는 말술방란없..
날날날건방란란 모이블방  없한심. 뇌러은수수목란란회.한한진 모"이점 방 없. .
다날날
9방방 방가 M었.

--- 다양성 =  1.0
--- 시드 = "이날 사건의 모든 방향이 원장 신연화"
이날 사건의 모든 방향이 원장 신연화는 찾저가병한을 마환있었없원 
했..퀴날날이점 었 신어화는 씨다. 뇌요수목술도란없
.날진연날 M목다. 뇌은수는 씨국간가있는는 퀴률높. 누신가화는 관건 퀴9며리 뇌건한 한공다. 뇌지수 공이란없있는 거현(블누누강신신화는는유저지가한가 밝
 밝어)는 어이 밝술방란려 .한한( 환말가 수목란마리없한. 한저'했

99방방방 없 "어 을 이리이신신화는 거저 간병  회한는 밝어히   이술방려회없한는 밝저블요  원을 거리없간   이점 회 퀴심. 강(("라환 말) 강한는 퀴어냐. 뇌윤수수도 씨지지.한진는 찾어수 밝 없지. 강날는 모목방방방 강한는 밝어현을 누 이신신화는 찾저병병병  거.. 강없신 .
99방방방방연 강어"현 말어 밝. 뇌한진는 밝어환)가리강는 M어O-에 찾한현  씨술도 지지한한진 ( 말환 뇌)강는 퀴양가리M' 회회한는 밝어블다  강신신화는 찾저병병한  거. 강없없  밝공심 강강는 M어-을  간한수 밝목은없한진 강 한어을 거률다높  강건 밝어저방 연 한 이었다" 강윤현  씨지. 지술 한한진

--- 다양성 =  0.5
--- 시드 = "날 사건의 모든 방향이 원장 신연화("
날 사건의 모든 방향이 원장 신연화(유유 가가병는 거어을을거거리간간  한이었( 누가가는는 퀴이점마  이현요 씨됐거. 뇌 병수도 거지한 연 관 이술도 없지심 공은다 M현" 찾한국 한어 병을 거아방간히  한라( 말을 수술방란국리한연화는 말저 밝술방 연화는 강저가 병을었거리 간다. 이날요신건 퀴저현  병병원을 거아. 강'신 .
99방방방없연 강 이블"어 밝목 밝마리없신' 퀴한(  찾병증 밝목.. 없없적 .
99방방연히  이블""라을 말심 .
러러 한한한는 말어었 을 거리.리'신라며었
9 누한가 M어밝을 거현다  이병는 밝률화는 강는 퀴현현  거지. 보은 수목도란공가..연 관  어방방을 없어며 강윤현  씨지지.한진에는 한현  씨 지지. 보분분분공공공 강강( 현현 누누지. 뇌심수술방란없적없연연라라 말어 수목 국회없있었 관 요양원원을 거리가리간'  공었

--- 다양성 =  1.0
--- 시드 = "날 사건의 모든 방향이 원장 신연화("
날 사건의 모든 방향이 원장 신연화(
유 가리병는 을리한' 회연가는 간찾수방방씨희(.말진희병
 원은 있리거보라날게다을 강윤심 병방퀴었 높' 현한. 찾어  원을 퀴률됐보률 씨 모수술방란없..라연라  이병방란회
없진었말( 뇌라다. 뇌심 수술도란모저지. M블M며 강에 신한수술

999방방방 강 모"라며 강윤현  씨지게 지술 한국 한  공"" 강 이 이었없"  뇌윤현  누지. 퀴은은살살 이이점 회회퀴었. 강이요신신화는 유저 병병병을 거아방간연 라  말술도 지지. 한진는 말어방을을 간한한라  이블도  없은". 강윤현  이점는  지심. 강신"현 찾저저병병병원을 거아. 간한   어심  강한는 밝어을 누 이신. 퀴었다. 강윤현  이이점방방 없한라며 강윤현  이이점신신화퀴한( 강간N  관양 국국한연 강관" 어을을을거거. 간히는 밝어을 거 간병는 밝양화는 씨저게

--- 다양성 =  0.5
--- 시드 = "드린 적 없는데. 게다가 MAO-A는"
드린 적 없는데. 게다가 MAO-A는 한한수  모이점방란란. M현는 M신화는 한저병병병 거.거. 간히. M현-O--에 신한현  지지지.강보 분분공공공 강병증 .
99방방 없 는 밝어히는 강 이 수술방란분리.리강 병관 밝원을 거리 간 밝. 뇌심도 한진는 밝어지을 뇌 수수도 회지있는 모었다. 모이점신방 퀴부는 강윤현  누이이었신신화퀴을(유라희희희는 말양 수술방란없없한. 라"""어 강 이한회회었한었 밝률.. 이신신화는 찾저가 병는 양양원을을거리간간간  한라" 말다  한윤현  씨지게 보심 분)공  강윤현 누 지현. 씨씨게게지현 씨게다. 강보는 밝한화는 누윤현며 씨게게지현 씨게게지한술보심보신화분분공공  강신며 퀴윤현현  누신OO-을에 한한는 밝어화 말누가..
 퀴은방살  이블수목 밝률없.   이점신었화퀴됐가 강는 이었수목방.란없리. M강-에게 퀴술방란

--- 다양성 =  1.0
--- 시드 = "드린 적 없는데. 게다가 MAO-A는"
드린 적 없는데. 게다가 MAO-A는 찾아  한건  모.심했 강증부적.
냐러없한라는 밝목점수마 이회있었는 회률요블N요 모심.증신퀴윤라마 한윤었 말 이씨퀴 없한며연 공현  누신신-는에 지지병 수술방란란없..
(9환)방방강연게신한화는 모심O수방회리없는 한진다  이날이란신는 퀴현며 강병양한진을(을 거저간간   한아저 퀴없""라며 간N 있신란회화(다. 강저병는 거저거을 한있는 요양원을 거

9러한한한는 말어양을을거거리간간'  한진( 모저가병병을 거률거간간   이었.
 뇌신화는 모저가병는 거양거을 간리 한라( 말희다. 이날점신  공저  병증 원을리거리 간'  한진((유환환가)가 M어-을 밝지다. 강신신신화는 씨저병지병병을 거리거리 연 분 밝공. 강한진는 어어을을 거리한진   수목도 씨지지.한라는 말윤현을 씨지지지한보보분

--- 다양성 =  0.5
--- 시드 = "고 물었다. 이에 신연화는 "저희 병"
고 물었다. 이에 신연화는 "저희 병병 원을 거리거간연 라 말 이점점  한어는 밝어다다 강뇌심수는 국심없있 관었 모요이방방없없한리리라희  수술도 국지지있한한라  말어 방회국없한연 M라" 밝다. 강한는 수술도란씨지지한한라  찾어을 밝률.한진진(("라환 말을 한한진(( 유환병)가 강는((현유 누누가게는현현 씨지지지보보보분분분공공공 강강(는 모현다방 이이점신  한현  강 지. 뇌에게한한현 씨씨지지지보보보분분분공공공 강강((유유희가강는는 이양수술방 지없한적연라( 말말 수목도 씨지지.한라현 밝어가 수목 씨마게.한진 (  )가 강관 (저현현 씨 지지지보보분분분공공공 강다. 퀴은은살살 이이점회회회었었었  이요요양원원을 거리거리 '  .

9러방없한  블이블점목 없어. 강리윤현  이신신신화는 거저가병병는 모양원을 거리강리 ( 유"라" 누 보 한지술 한국있는

--- 다양성 =  1.0
--- 시드 = "고 물었다. 이에 신연화는 "저희 병"
고 물었다. 이에 신연화는 "저희 병병지 수어방. 한뇌증 밝목밝목살없씨도 -퀴가리어'한(라"현 말병다.
러했한.한 한는 밝공거. 간 없.이란없은점M졌 N심했심.

99방방방 모있유이었요간없'었 어저리 간한 라 씨다.
란은화는국을어N률말저방보었 . 밝
뇌적.날요 어신방을리없률리높' 병관원을 모률다. 강진이는 어현을 씨리건지신라보분밝분공강연강강는는어을현란) 요 한 연 회리한병병 을찾거. 간리한는 라현  씨윤리
됐. 뇌지한살한 화(유.-뇌신병는한밝아 가 보현 유가리가심요관병원을양거국리간리 병병한원을 거현 높간 관현 (( 공방희연 